# Incremental Dynamic Learning

## Overview

The idea behind Incremental Dynamic Learning networks is that information is not always learned at once, up front. As information is encountered, we train on the new data. Adding the new information to our existing knowlege base. Humans, are said to be able to recognize an image in 13 milliseconds after seeing it.The new image is added to their existing learned information and they do not have to retrain on all images they have ever seen.

## Example

As an example of how this can be used in the real world, I will use field biologists for my use case.

Imagine a group of field biologists are in the field and all have a mobile application which allows them to photograph wildlife they encounter, label it and forward it to a central system. The central system takes in the new photographs and trains a neural network on each piece of new data that comes in. W/out having to retrain a new model based on all past photographs. 

## Experiment 1

The experiment will consist of using a pre-trained network from TensorFlow (VGG) to generate our classification data. Our data will consist of using flower images provided with TensorFlow examples.

Once the data has be converted by the VGG network, the process of training our network begins. This will be done by feeding the network 1 image at a time, allowing the network to train for a short duration (13ms) and then move onto the next image. A fixed validation set will be used to monitor progress as the images are fed one at a time to the network.

The goal is to get an acceptable accuracy, similar to what a traditionally trained network would achieve.

## Experment 2

For the second half of this experiment will be to start with a single classification item 'UNKNOWN' and grow the classification set as needed. This will be done by checking a classificaiton index dictionary for the incoming label. If the label does not exist, then a prediction will be made with the current network. The classifcation that matches the image the best will be used to prime the initial weights of the new node to be added to the classifcation set. Once the node is added, the normal training on the image will be performed.

## Structure

The base for this experiment will be from a lesson given by Udacity on Transfer Learning. The first part of this lesson uses VGG to generate the training data for a flower classifcation system. I will use the output from that lesson to provide the training for this Expermiment.

## Get libraries and data

In [3]:
import os

import numpy as np
import tensorflow as tf

In [4]:
# read codes and labels from file
import csv

with open('labels') as f:
    reader = csv.reader(f, delimiter='\n')
    labels = np.array([each for each in reader if len(each) > 0]).squeeze()
with open('codes') as f:
    codes = np.fromfile(f, dtype=np.float32)
    codes = codes.reshape((len(labels), -1))

In [5]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
lb.fit(labels)

labels_vecs = lb.transform(labels)

In [6]:
from sklearn.model_selection import  train_test_split

X_train, X_test, y_train, y_test = train_test_split(codes, labels_vecs, train_size=0.8, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [7]:
print("Train shapes (x, y):", X_train.shape, y_train.shape)
print("Validation shapes (x, y):", X_val.shape, y_val.shape)
print("Test shapes (x, y):", X_test.shape, y_test.shape)

Train shapes (x, y): (2642, 4096) (2642, 5)
Validation shapes (x, y): (294, 4096) (294, 5)
Test shapes (x, y): (734, 4096) (734, 5)


## Construct the network

In [2]:
from tensorflow.contrib.keras import models
from tensorflow.contrib.keras import layers
from tensorflow.contrib.keras import optimizers

def build_flower_classifcation_model(learning_rate=0.001):

    # Power ball powerball numbers model
    model = models.Sequential()
    model.add(layers.Dense(256, input_shape=[codes.shape[1]]))
    model.add(layers.Dense(labels_vecs.shape[1], 
                           kernel_initializer='zeros', 
                           bias_initializer='zeros'))

    model.add(layers.Activation('softmax', name='out'))
    optimizer = optimizers.Adam(lr=learning_rate)

    model.compile(optimizer=optimizer,
                  loss='mean_squared_error',
                  metrics=['accuracy'])
    model.summary()
    return model

In [8]:
model = build_flower_classifcation_model(learning_rate=0.0001)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               1048832   
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 1285      
_________________________________________________________________
out (Activation)             (None, 5)                 0         
Total params: 1,050,117
Trainable params: 1,050,117
Non-trainable params: 0
_________________________________________________________________


## Train the model 1 image at a time

In [9]:
for index in range(len(X_train)):
    single_image = X_train[index:index+1]
    single_label = y_train[index:index+1]
    if index % 10 == 0:
        verbose = True
    else:
        verbose = False
    
    print ('Image: ', index)
    model.fit(single_image, single_label, validation_data=(X_val, y_val), batch_size=1, epochs=2, verbose=verbose)
    
print (model.evaluate(X_test, y_test))

Image:  0
Train on 1 samples, validate on 294 samples
Epoch 1/2
1/1 [==============================] - 0s - loss: 0.1600 - acc: 0.0000e+00 - val_loss: 0.1605 - val_acc: 0.1395
Epoch 2/2
1/1 [==============================] - 0s - loss: 0.1476 - acc: 1.0000 - val_loss: 0.1613 - val_acc: 0.1395
Image:  1
Image:  2
Image:  3
Image:  4
Image:  5
Image:  6
Image:  7
Image:  8
Image:  9
Image:  10
Train on 1 samples, validate on 294 samples
Epoch 1/2
1/1 [==============================] - 0s - loss: 0.1376 - acc: 1.0000 - val_loss: 0.1478 - val_acc: 0.4354
Epoch 2/2
1/1 [==============================] - 0s - loss: 0.1338 - acc: 1.0000 - val_loss: 0.1472 - val_acc: 0.4184
Image:  11
Image:  12
Image:  13
Image:  14
Image:  15
Image:  16
Image:  17
Image:  18
Image:  19
Image:  20
Train on 1 samples, validate on 294 samples
Epoch 1/2
1/1 [==============================] - 0s - loss: 0.1786 - acc: 0.0000e+00 - val_loss: 0.1333 - val_acc: 0.4898
Epoch 2/2
1/1 [==============================] - 